## <span style="color:green">Import the packages/modules required for this exercise</span>

We need the following packages: hydrofunctions (hf), pandas (pd), numpy (np), pyplot (plt) from matplotlib, stats (sp) from scipy, seaborn and probscale. The paranthesis contains the commonly used short forms for these libraries.

In [ ]:
# Important Packages required for the analysis
import hydrofunctions as hf
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sp

import sys
## To add the path to the packages available with Python 3 kernel (when working in Python [conda env:qgis] kernel)
sys.path.append('/apps/share64/debian7/anaconda/anaconda3-5.1/lib/python3.6/site-packages/')

## To add the path to the packages available with Python [conda env:qgis] kernel (when working in Python 3 kernel)
sys.path.append('/apps/share64/debian7/anaconda/anaconda3-5.1/envs/qgis/lib/python3.7/site-packages/')

# For plotting in probscale
import seaborn
import probscale

## <span style="color:green">USGS Station and Analysis Period</span>

We need three inputs from the user for carrying out the analysis:
<ol type="1">
    <span style="color:red"><li> USGS station code</li></span>
    <span style="color:red"><li> Analysis start date </li></span>
    <span style="color:red"><li> Analysis end date </li></span>
    
<p style='text-align: justify;'><b><u> Note:</u></b> Both dates value should be entered in YYYY-MM-DD format (including the hyphen symbol). <br>
<br>
To check whether your code is working correcly, let us use  the USGS gage on Wabash River at Lafayette, IN (<b>03335500</b>) for a sample analysis period from <b>2017-01-01</b> to <b>2017-05-31</b>.</p>

In [ ]:
## Input the required USGS Station and Analysis Period
# WRITE A CODE TO DEFINE VARIABLES FOR STATION NUMBER,START DATE AND END DATE
USGS_Station = input(str("Enter the USGS Station Code: \n"))
Start_Date = input(str("Enter the starting date (in YYYY-MM-DD format): \n"))
End_Date = input(str("Enter the ending date (in YYYY-MM-DD format): \n"))

## <span style="color:green">Obtain the data using hydrofunction from USGS NWIS</span>

One of the two options below may be used to obtain the daily value "dv" of the streamflow

#### <span style="color:red">Option01</span>
<code>data = hf.NWIS(USGS_StationCode,'dv',period='P365D') # P365D indicates past 365 days</code>

#### <span style="color:red">Option02</span>
This more useful as discharges for desired period can be obtained and let us use this option in this tutorial. <br>
<code>data = hf.NWIS(USGS_StationCode, 'dv', Start_Date,End_Date)</code>

Finally, run <code>data.get_data()</code> to download the daily data from USGS.

In [ ]:
#WRITE THE CODE TO DOWNLOAD DAILY STREAMFLOW FROM USGS USING HYDROFUNCTION
data = hf.NWIS(USGS_Station, 'dv', Start_Date,End_Date)
data.get_data()

## <span style="color:green">Data Checking</span>

<p style='text-align: justify;'>Now, let us check the data by printing the first n rows (default: 5) using the head statement. This is helpful for quickly testing if your object has the right type of data in it.</p>

<u><b>Note:</b></u>
    
00060 is parameter code for Discharge (cfs)<br>
00003 is the stat code for daily mean

In [ ]:
print(data.df().head())
print("\n")

## <span style="color:green">Dataframe and Column name change</span>

<p style='text-align: justify;'>We get the output from the hydrofunctions as "class function". By defining a new dataframe, let us store the data into pandas dataframe to do the manipulations. In the above data, we have two columns namely Discharge and Qualification Code. Let us rename the default column names and drop the qualification code column.</p>


In [ ]:
strflow=pd.DataFrame(data.df())
strflow.columns=('Discharge','QC')
strflow=strflow.drop(columns='QC')

## <span style="color:green">Discharge hydrograph</span>


Now, we will plot the discharge hydrograph for the analysis period.

In [ ]:
#WRITE YOUR CODE HERE TO PLOT THE DISCHARGE HYDROGRAPH
#ALSO PROVIDE AXIS TITLES WITH UNITS

file_name1 = './DisHyd_' + USGS_Station + '.png'
strflow.plot()
plt.tight_layout()
plt.xlabel('Time (days)')
plt.ylabel('Discharge (cfs)')
plt.title('Discharge Hydrograph for USG Station: '+ USGS_Station +'\n (Period:' + Start_Date + 'to' + End_Date + ')\n')
plt.tight_layout()
plt.savefig(file_name1)

## <span style="color:green">Calculation of Exceedance Probability</span>

<p style='text-align: justify;'>Now, we have to calculate the exceedance probability using the Weibull (1939) plotting position formula. The steps involved in the process are outlined earlier.</p> 

In [ ]:
## Calculation of exceedance probability
# WRITE YOUR CODE HERE

# Step 1: Sort the data in descending order
flow = strflow.sort_values('Discharge',ascending = False)
print(flow.head())

# Step 2: Calculating the rank and reversing the rank to give the largest discharge a rank of 1
ranks1 = sp.rankdata(flow, method = 'average')
ranks2 = ranks1[::-1]

# Step 3: Calculate the exceedence probability
exceed_prob = [100*(ranks2[i]/(len(flow)+1)) for i in range (len(flow))]


## <span style="color:green">Flow Duration Curve in linear-linear scale</span>

<p style='text-align: justify;'>Let us plot the flow duration curve with discharge in linear scale and exceedance probability in linear scale.</p> 

In [ ]:
# Step 4: Plot the FDC in different scale
## Flow Duration Curve
## Discharge in linear-linear scale
# WRITE YOUR CODE HERE

# To generate file name as per station number
file_name2 = './FDC_Lin_Lin_'+USGS_Station+'.png'

plt.figure()
# plt.scatter(prob,flow)
plt.plot(exceed_prob,flow)

## Add grid lines to both axes
plt.grid(which = 'both')
plt.xlabel('Exceedance Probability (%)\n[in linear scale]')
plt.ylabel('Discharge (cfs)\n [in linear scale]')
plt.xticks(range(0,100,5))
plt.title('Flow Duration Curve for USGS Station: ' + USGS_Station + '\n (Period: ' + Start_Date + 'to' + End_Date + ')\n [linear-linear scale]')
plt.tight_layout()
plt.savefig(file_name2)

## <span style="color:green">Flow Duration Curve in log-linear scale</span>

<p style='text-align: justify;'>Let us plot the flow duration curve in with discharge in logarithmic and exceedance probability in linear scale.</p> 

In [ ]:
## Flow Duration Curve
## Discharge in log-linear scale
## Variables (prob & flow) calculated above used for plotting
# WRITE YOUR CODE HERE

file_name3 = './FDC_Log_Lin_'+USGS_Station+'.png'

plt.figure()

# plt.scatter(prob,flow)
plt.plot(exceed_prob,flow)
plt.yscale('log')

# Add grid lines to both axes

plt.grid(which = 'both')
plt.xlabel('Exceedance Probability (%) \n[in linear scale]')
plt.ylabel('Discharge (cfs) \n [in logarithmic scale]')
plt.xticks(range(0,100,5))
plt.title('Flow Duration Curve for USGS Station: ' + USGS_Station + '\n (Period: ' + Start_Date + 'to' + End_Date + ')\n [log-linear scale]')
plt.tight_layout()
plt.savefig(file_name3)

## <span style="color:green">Flow Duration Curve in log-probability scale</span>

<p style='text-align: justify;'>Let us change the linear scale of discharge to logarithmic scale for better visualisation at the two ends of the probability axis (i.e. extreme values) exceedance probability in probability scale (instead of linear scale).</p> 

In [ ]:
## Flow Duration Curve
## Discharge in log-probability scale
## Variables (prob & flow) calculated above used for plotting

file_name4='./FDC_Log_Prob'+USGS_Station+'.png'
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(exceed_prob,flow)
ax.set_ylim(0.1, 10000)
ax.set_yscale('log')
ax.set_title('Flow Duration Curve for USGS Station: '+ USGS_Station +'\n (Period:' + Start_Date + ' to ' + End_Date +')\n[log-probability scale]' )
## Add grid lines to both axes
plt.grid(which = 'both')
plt.xlabel('Exceedance Probability (%) \n [in probability scale]')
plt.ylabel('Discharge (cfs) \n [in logarithmic scale]')
ax.set_xlim(0.5, 99.5)
ax.set_xscale('prob')

seaborn.despine(fig=fig)
plt.tight_layout()
plt.savefig(file_name4)